### 03_Feature_Engineering

In [1]:
#Imports & Config
import holidays
import pandas as pd
import numpy as np
import gc

pd.set_option("display.max_columns", 100)

##### Load Data

In [3]:
df = pd.read_parquet("data/cleaned/yellow_taxi_clean_2025.parquet")

df = df.rename(columns={
    'tpep_pickup_datetime': 'pickup_datetime'
})

df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df = df.sort_values('pickup_datetime')

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42018630 entries, 7495304 to 41144824
Data columns (total 24 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   pickup_datetime        datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     int64         
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee        

##### Create Hourly Timestamp

In [5]:
df['pickup_hour_ts'] = df['pickup_datetime'].dt.floor('h')

##### Aggregate to Hour × Zone Level

In [7]:
hourly_demand = (
    df
    .groupby(['PULocationID', 'pickup_hour_ts'], observed=True)
    .agg(
        demand=('VendorID', 'count'),
        avg_fare=('fare_amount', 'mean'),
        avg_trip_distance=('trip_distance', 'mean'),
        avg_passenger_count=('passenger_count', 'mean')
    )
    .reset_index()
)

hourly_demand.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1272868 entries, 0 to 1272867
Data columns (total 6 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   PULocationID         1272868 non-null  int32         
 1   pickup_hour_ts       1272868 non-null  datetime64[us]
 2   demand               1272868 non-null  int64         
 3   avg_fare             1272868 non-null  float64       
 4   avg_trip_distance    1272868 non-null  float64       
 5   avg_passenger_count  1272868 non-null  float64       
dtypes: datetime64[us](1), float64(3), int32(1), int64(1)
memory usage: 53.4 MB


##### Free Memory

In [9]:
del df
gc.collect()

29

##### Time-based Features

In [11]:
hourly_demand['hour'] = hourly_demand['pickup_hour_ts'].dt.hour.astype('int8')
hourly_demand['weekday'] = hourly_demand['pickup_hour_ts'].dt.weekday.astype('int8')
hourly_demand['month'] = hourly_demand['pickup_hour_ts'].dt.month.astype('int8')

hourly_demand['is_weekend'] = hourly_demand['weekday'].isin([5, 6]).astype('int8')
hourly_demand['is_rush_hour'] = hourly_demand['hour'].isin([7, 8, 9, 16, 17, 18]).astype('int8')


##### Cyclical Encoding

In [13]:
hourly_demand['hour_sin'] = np.sin(2 * np.pi * hourly_demand['hour'] / 24)
hourly_demand['hour_cos'] = np.cos(2 * np.pi * hourly_demand['hour'] / 24)

hourly_demand['weekday_sin'] = np.sin(2 * np.pi * hourly_demand['weekday'] / 7)
hourly_demand['weekday_cos'] = np.cos(2 * np.pi * hourly_demand['weekday'] / 7)


##### Holiday Features (US / NYC)

In [15]:
import holidays

us_holidays = holidays.US(state='NY')

hourly_demand['is_holiday'] = (
    hourly_demand['pickup_hour_ts']
    .dt.date
    .isin(us_holidays)
    .astype('int8')
)


##### Sort Before Lag Features

In [17]:
hourly_demand = hourly_demand.sort_values(
    ['PULocationID', 'pickup_hour_ts']
)


##### Lag Features

In [19]:
lags = [1, 2, 3, 24]

for lag in lags:
    hourly_demand[f'demand_lag_{lag}'] = (
        hourly_demand
        .groupby('PULocationID')['demand']
        .shift(lag)
    )


##### Rolling Features

In [24]:
windows = [3, 6, 24]

for window in windows:
    hourly_demand[f'demand_roll_mean_{window}'] = (
        hourly_demand
        .groupby('PULocationID')['demand']
        .shift(1)
        .rolling(window)
        .mean()
    )


##### Drop NaNs (from lags & rolling)

In [26]:
hourly_demand = hourly_demand.dropna().reset_index(drop=True)

hourly_demand.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1266674 entries, 0 to 1266673
Data columns (total 23 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   PULocationID         1266674 non-null  int32         
 1   pickup_hour_ts       1266674 non-null  datetime64[us]
 2   demand               1266674 non-null  int64         
 3   avg_fare             1266674 non-null  float64       
 4   avg_trip_distance    1266674 non-null  float64       
 5   avg_passenger_count  1266674 non-null  float64       
 6   hour                 1266674 non-null  int8          
 7   weekday              1266674 non-null  int8          
 8   month                1266674 non-null  int8          
 9   is_weekend           1266674 non-null  int8          
 10  is_rush_hour         1266674 non-null  int8          
 11  hour_sin             1266674 non-null  float64       
 12  hour_cos             1266674 non-null  float64       
 1

##### Save Feature Dataset

In [28]:
hourly_demand.to_parquet(
    "data/cleaned/yellow_taxi_features_hourly_zone_2025.parquet",
    index=False
)
